# A Brief Peek into the Chess Scene in America using Data Science!

by Michael Omori

Here I present my work and how I went about it. I also have slides I presented for anybody who wants to cut to the chase. Or you can read my conclusion. My repo is a jungle, so I just copy snapshots here, also so I don't have to rerun code. If you see anything I missed and/or want to contact me, you can email me at omorim@hawaii.edu. Code is here: https://github.com/AstroBoy1/chess-ds

## Table of Contents

Chess History <br>
My own journey in chess and data science <br>
Questions <br>
Architecture Flowchart <br>
Scraping <br>
Database <br>
Cleaning <br>
Analysis and Visualizations <br>
Modeling <br>
Conclusions <br>
Future Work <br>
Appendix

## Chess History

A long history. You can read wikipedia: https://en.wikipedia.org/wiki/Chess. <br>
Essentially believed to have originated in India in c. 280-550. <br>
The modern rules came about around the 1200s. <br>
1886 was the first official world chess championship won by Wihelm Steinitz. <br>
Lately chess has been dominated by Magnus Carlsen. <Br>
With computers really making breakthroughs from IBM's deep blue beating Kasparov in 1997 to DeepMind's AlphaZero in 2017 which has a rating of somewhere around 3800.

## My own journey in chess and data science

Where to start. I'll post more on https://michaeljomori.com/ if you're interested. Born in Seattle, Washington, I've been playing since I was around 2 years old thanks to my dad. Won some state and nationals tournaments. Coaching chess now and doing my master's in computer science. Worked for a year after undergrad as a software engineer in the data science division. So hopefully below I can provide you with some interesting insights.

## Architecture Flowchart

Here is my architecture flowchart.

<img src="images/architecture.png" width="600">

## Scraping

I get user data from uschess.org Below is an example. I use requests for getting HTML and BeautifulSoup for parsing it.

<img src="images/user_site.png" width="600">

<img src="images/user_site2.png" width="400">

My UID is 12852397 with a link of http://www.uschess.org/msa/MbrDtlMain.php?12852397. We can get another user's information by replacing my UID with their UID like http://www.uschess.org/msa/MbrDtlMain.php?11111111. This is a non-existent user. I don't have a list of UIDs so I do guess and check. I try binary search on the UIDs assuming there is some range of values and continuity to get a lower and upper bound

The lowest id: 12435021 <br>
The highest id: 12939951 <br>
Then I do random sampling without replacement when fetching the URLs. You may have noticed my overall ranking is out of 67901, which means there are 67901 active chess players. The difference in highest id and lowest id on the otherhand is 504,930. But not all of these links actually worked.

Scraping events was also uncertain in that I don't know how many tournaments a user has played in. <br>
Tournaments can span multiple pages like such: <br>
http://www.uschess.org/msa/MbrDtlTnmtHst.php?12852397.2 <br>
http://www.uschess.org/msa/MbrDtlTnmtHst.php?12852397.3 <br>
So I just keep scraping until I hit some empty event using a while loop and have it return a non-zero value in certain cases. I also wrote my own get_event_ratings function and you can look at my github code if you're interested.

<img src="images/event_loop.png" width="600">

I do some manual parsing, but also do some regex like such:

<img src="images/regex.png" width="600">

Here is an example of what the events looks like before I transform it.

<img src="images/events_df.png" width="600">

## Database

I use SQLITE for my backend database. SQL Server would be overkill. Below is my basic star schema after cleaning and transformations. I start with a users table, and events table. Then do this to it. No need for snowflake schema. On top of which I create a few views for easier querying

<img src="images/schema.png" width="600">

Users contains: UID, State, Gender <br>
Fact table contains: UID, event_id, rating_before, rating_after <br>
Events_dim_cleaned contains: event_id, event_name, date, state <br>
uid_date_ratings: uid, 1991, 1992, ..., 2019 <br>
I use text for many of these attributes because sometimes the values will be "NA" or have a string in front of a number like 1000 P(5). So it would be nice to use like an int for example, but this fails a lot. Each table has one primary key and foreign key. And I chose to do inner joins to reduce my data a bit.

Data is 283,785 users who've played in at least one tournament. 488, 586 people who are registered with USCF. 3,283,074 million events in the fact table. Events are redundant originally when collecting data, so I make a dim table for it called events_dim_cleaned. There are 199,129 unique events. I only look at people who have played in at least one tournament in this notebook, but may take a look at everyone in a future one.

## Cleaning

Despite carefully scraping the data, I still get inconsistencies due to the website and my own mistakes. So I have to do some cleaning. This includes some times where I get fields mixed up like the state field ended up overlapping with the event field. Also when I create some tables, there is missing data. Such as most of the time, I scraped blitz ratings, but usually the person was not playing in a blitz tournament. Of course, during my scraping process I had to make many passes through to get the fields reasonably correctly because of the lack of documentation. Some like state fields I manually found and handcoded the offset, for others I used regex to look for parentheses or where I think the field should begin.

## Analysis and Visualizations

Believe it or not, a few of these visualizations had me scratching my head on how to make: I had to create views and manipulate the data, which was sort of big in the millions. I will mention some things I did, you can look at my code for more deets. There were probably several ways of creating these plots, and each would've gotten slightly different results. I used matplotlib, seaborn, and good o'l Tableau. To create the graph below, I make a new table where there is one uid column, and the rest of the columns are years. I take the rating of each user as of December and use it for that rating in that year.

<img src="images/number_of_chess_players.jpg" width="1000">

Chess has seen an increase in popularity from 1991-2004 at about a rate of 20,000 new people registering each year. But then a sudden plateau occurs afterwards. I'm not sure exactly what the reason for this is, as I would think chess is getting more popular. Maybe the most probable reason is I messed up this graph. So either during the data collection process, I didn't get all the data and get it correctly, or while doing the analysis I messed up. Like maybe I was inner joining my users table and fact table inccorrectly. If I had left outer joined, maybe it could look more reasonsable if I had messed up getting the events for the users. But for now, we'll have to assume the validity of my data and methods. Please bear with me. One more thing is unknown: which is really just the gender is unknown. It could be non-male and non-female, or just the person didn't put in their gender. Below I calculate gender percentages.

<img src="images/gender_ratios.png" width="1000">

Gender percentages have remained roughly the same with chess being a male dominated sport at around 80%. Next I look at the max rating per year.

<img src="images/max_rating_per_year.jpg" width="1000">

The maximum rating for male is the maximum for all, which was acheived by Magnus Carlsen, with a rating of 2882. In this graph, male is under the all. The highest female rating acheived was by Judit Polgar with a rating of 2735. Next I plot the distributions of ratings.

<img src="images/rating_distributions.jpg" width="1000">

Here is the distribution of ratings using histograms and also KDEs. Each of them tends to have a bell-shaped curve; however, there is a peak at 100, possibly because I only include people who play in at least one tournament and there is a lot of people who play in only one tournament and don't play in any more, so there rating stays low. I also graph the average ratings per by gender.

<img src="images/average_rating_per_year.jpg" width="1000">

The most interesting trend is the decreasing rating to an average of a little over 800. From my own experiences, I can only imagine this is the case because of the inlfux of children playing chess tournaments who play a bit when they are young and then stop playing in tournaments as they get older.The stabilization around 2004 is supported by the fact by the number of chess players who are registering at that point has also stabilized.  To determine if chess has seen an increase in popularity, I took a look at the number of chess players in each state. 

<img src="images/Dashboard_Players_by_Location.png" width="800">

New York hs the highest with 36,401, followed by California, and Texas. Along with Florida and Illinois. Some of the states with not too many people include the mountainous regions such as Idaho and Wyoming. Another interesting one is West Virginia, as pointed out by Professor Mahdi Belcaid. It is surrounded by states that have a fair amount of chess players, but it did not want to follow suit. Next, I figure having a lot of tournaments is also something a chess player wants. So here's the graph. 

<img src="images/Dashboard_Tournaments_by_Location.png" width="800">

It looks similar to the one above. With New York, California, and Texas being hotbeds for tournaments. While West Virginia is lackadasical once again. An interesting difference is that there is some more uniformity among the states as there are no states that have very few chess tournaments. I then graph the two against each other to get the plot below.

<img src="images/events_chess_players_state.jpg" width="1000">

Because of the linear looking trend, I calculated the correlation and got 0.96 linear pearson's correlation, suggesting high linearity between the two. Thus the number of chess tournaments and the number of chess players in each state is highly correlated. Which is intuitive.

## Modeling

### Predicting Chess Rating (Supervised Regression)

I used Gender (categorical), State (categorical), and Number of Chess Tournaments (numerical) played for the features. <br>
I used a Catboost model. <br>
Data is 283, 785 users. <br>
RMSE average mean across 5 folds, 80:20 split, 371.16 on training, 371.24 on validation <br>
STDV: 16.890 <br>
RMSE on test is 371.56
With a reminder that the scale of ratings is from 0-2900 for humans.

### Feature Importance

Features are only so useful as the power of the model. Which in this case, is so so, so the below can be taken with a grain of salt: <br>
1. Number of tournaments played: 53.119 <br>
2. Location: 23.572 <br>
3. Gender: 23.310

Number of tournaments played, which is linked with experience is the highest predictor of chess rating. So someone who has never played in a chess tournament vs someone who has played in a lot of chess tournaments will have very different ratings. The next two are about equal according to the model and they are location and gender. Location plays a slightly higher role than gender.

## Conclusions

Chess has had a rich history since around c. 280, and some people thought chess has reached it’s peak, but computers have proved otherwise with the advent of AlphaZero. I love chess and wanted to see if I could answer some questions about chess using my data science toolkit. I used a standard data science architecture: scraping data using VMs into a SQLite database, exploring and cleaning it, analyzing and modeling the data, then creating visualizations and insight. With an iterative approach to the process.

Some of the most interesting things I found in my opinion are that chess has been seeing an increase in popularity, but since 2004, it has not been as much as from 1991. As expected, about 80% of chess players are male, and all of the outreach has not done much to change this. There are more men who are better than chess at women. Largely in part, because there are more men than women. This does not suggest women are inherently worse than women at chess. Just that there is more variation among men because of the higher number of male players. Next time, I may try some significance testing for this. The average rating is about 800, which is to be expected. But it would be interesting too look at people who are active players, that way we get rid of people who aren’t serious about chess. New York, California, Texas, Illinois, and Florida are great places to live in for chess players. Which many people may have intuitively known already. But most people have probably not seen the map I made that shows chess popularity across all of the states.

If anything, this shows that there is a ton that can be done using chess data and there will be more to come from me!


## Future Work

There are several areas of interest for me. Some include looking at player vs. player statistics. Like how likely is one player to beat another player. Also looking at some of the top players and how fast they improved their rating. Maybe make my Tableau dashboards public so people can interact with them. Overall, I may ask USCF for the complete picture, so all of their data, because I don't think I got quite all of the data. I can also try using outer joins instead of inner joins and see how things go. Lastly, I want to get the counts of the bins for when I created the histograms and match it up with the chess player rating categories like D, C, B, A, Expert, Master, etc. to explicitly get the number of chess players in each rating bin.

## Appendix

For those interested in more details, I include some...

### Google Cloud VM

<img src="images/vm_1.png" width="400">

<img src="images/vm_2.png" width="400">

I use Debian 9 for my OS because I like the customizability on it. And a persistent 100GB SSD storage system because it’s faster.
I use a VM because it can stay on forever and I can use it just for this project. I use Google Cloud because it integrates well with Google Colab and Drive. And they provide some free credit.

### Code Structure

My code is here: https://github.com/AstroBoy1/chess-ds

I have a tests folder with simple unit tests. I have a docs folder with a doc to be updated. My main file is scrape_insert.py. Which is a few hundred lines of code. It calls py_sql.py at one point to create a sqlite database and create tables. I have a requirements file for the package versions that need to be installed to prevent any conflicts. I use the standard main import. read_sql.py is used for reading from the db mostly for testing purposes. rating_model.ipynb contains the Catboost code. I do some exploration and analysis in chess_exploration.ipynb. I do Tableau stuff in chess.twb. I have some sql files where I create tables and views.

### Error Checking

<img src="images/error_checking.png" width="400">

Above is an example of error checking and logging, I do this for my scraping code to ensure it doesn't stop and to make debugging easier.

### Progress Bar

<img src="images/progress_bar.png" width="400">

Progress bar was useful as my vm was running for a month, and failed a few times.

### Median rating

<img src="images/Median_rating_per_year.jpg" width="1000">

Median looks similar to the average at first glance.

# Thank you for reading!